In [32]:
CLASS_LABELS_FLAT = "../assets/labels_final.json"  # provided file for label info
AGG_AVFEAT = "../models/agg_avfeature.pkl.gz"             # custom file for merged audio and video features
WORKSHOP_BASE = "https://vmlr-workshop.STORAGE"
LABEL_CSV = "../models/class_labels.csv"
FEATURE_H5 = "../models/class_feature_{feature}.hdf5"

import pandas as pd
from pathlib import Path

df_labels = pd.read_json(CLASS_LABELS_FLAT)
print(df_labels)

# df_labels2 = pd.read_pickle(AGG_LABELS)
df_feat = pd.read_pickle(AGG_AVFEAT)
print(df_feat)

                         etag                                  labels  \
0    a039cb37f290fe4a4127bbd2  [holiday, gift giving, family moments]   
1    b6dbe02fdf00f08a61a44b8f                  [holiday, gift giving]   
2    406f92c5353fc790dc510494             [halloween, family moments]   
3    0ee359a9827d30449f63f088              [holiday, shopping scenes]   
4    7127fc69efb16a612a17d43d                        [family moments]   
..                        ...                                     ...   
995  450ff3ffdcfe6b758545820a                        [family moments]   
996  ac519b1836d6d8b939ebf0d2                     [none of the above]   
997  3609fc944d92df81d2190856                  [holiday, gift giving]   
998  10a4eb447f297a5727e4536b                             [halloween]   
999  c7cf0e1219f109f1b9f60207                               [holiday]   

                                                  data  
0    https://vmlr-workshop.STORAGE/gifts/v...  
1    https://vmlr-

In [35]:

df_labels_out = df_labels.explode("labels").rename(columns={"labels":"label"}).drop(columns=["etag"])
df_labels_out["data"] = df_labels_out["data"].apply(lambda x: x.replace(WORKSHOP_BASE+"/", ''))
df_labels_out["data"] = df_labels_out["data"].apply(lambda x: x.replace('/', '_'))
print(df_labels_out)
df_labels_out[['data', 'label']].to_csv(LABEL_CSV, index=False, header=False)


                 label                                     data
0              holiday   gifts_vid_gift_give_take_8-4-of-12.mp4
0          gift giving   gifts_vid_gift_give_take_8-4-of-12.mp4
0       family moments   gifts_vid_gift_give_take_8-4-of-12.mp4
1              holiday  gifts_vid_gift_give_take_28-7-of-16.mp4
1          gift giving  gifts_vid_gift_give_take_28-7-of-16.mp4
..                 ...                                      ...
996  none of the above              xmas_vid_xmas_8-7-of-49.mp4
997            holiday   gifts_vid_gift_give_take_0-6-of-11.mp4
997        gift giving   gifts_vid_gift_give_take_0-6-of-11.mp4
998          halloween   halloween_vid_halloween_3-25-of-68.mp4
999            holiday  gifts_vid_gift_give_take_0-10-of-11.mp4

[1525 rows x 2 columns]


In [38]:
import h5py
from pathlib import Path

for feat_type in ["combined", "audio", "video"]:
    path_feat = Path(FEATURE_H5.format(feature=feat_type))
    if path_feat.exists():
        path_feat.unlink()
    with h5py.File(str(path_feat),'w') as f:
        for idx,row in df_feat.iterrows():
            path_clean = row['asset'].replace("/", '_')
            try:
                f.create_dataset(path_clean, data=row[feat_type])
            except RuntimeError as e:
                print(f"Skipping asset '{path_clean}', due to error {e}")
    print(f"Wrote to type {feat_type} into file '{FEATURE_H5}'...")



Skipping asset 'gifts_vid_gift_give_take_11-2-of-13.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_11-2-of-13.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_11-2-of-13.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_3-10-of-10.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_3-10-of-10.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_3-10-of-10.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_3-10-of-10.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_3-10-of-10.mp4', due to error Unable to create link (name already exists)
Skipping asset 'gifts_vid_gift_give_take_3-10-of-10.mp4', due to error Unable to

Wrote to type video into file '../models/class_feature_{feature}.hdf5'...
